# Jupyter Notebook example for MERRIN

In [ ]:
# ==============================================================================
# Imports
# ==============================================================================
from typing import Literal
from pandas import DataFrame

from merrin import MerrinLearner, Observation, MetabolicNetwork

## Inputs

### Mandatory

In [ ]:
sbml: str = 'ecoli-small/metabolic_network.sbml'
objective: str = 'Growth'
pkn: str = 'ecoli-small/pkn.txt'
observations_json: str = 'ecoli-small/timeseries_kft.json'

### Optional

In [ ]:
# ~ Solving modes
optimisation_mode: Literal['all', 'subsetmin'] = 'all'
projection_mode: Literal['network', 'node'] = 'network'

# ~ Solving parameters
lpsolver: Literal['glpk', 'gurobi'] = 'glpk'
nbsol: int = 0 # 0 to enumerate all solution, else the nb of solution to enum
timelimit: float = -1 # -1 if not timelimit, else the timelimit value in second
max_gap: int = 10 # maximum number of timestep than can be added
max_error: float = 0.1 # maximum error rate between observations and predictions
max_clause: int = 20 # maximum number of clauses per rules in DNF

## Preprocessing

### Parse the Prior Knowlege Network file

In [ ]:
parsed_pkn: list[tuple[str, int, str]] = []
with open(pkn, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        line = line.strip()
        u, s, v = line.split('\t')
        parsed_pkn.append((u, int(s), v))

### Parse Observation json 

In [ ]:
observations: list[Observation] = Observation.load_json(observations_json)

### Parse the SBML

In [ ]:
mn: MetabolicNetwork = MetabolicNetwork.read_sbml(sbml)

## MERRIN

In [ ]:
learner: MerrinLearner = MerrinLearner()
learner.load_instance(mn, objective, parsed_pkn, observations)

rules_df: DataFrame

In [ ]:
if projection_mode == 'network':
    bns: list[list[tuple[str, str]]] = learner.learn(
        nbsol=0, display=True, lp_solver=lpsolver, max_clause=max_clause,
        max_error=max_error, max_gap=max_gap, timelimit=timelimit,
        subsetmin=optimisation_mode == 'subsetmin'
    )
    rules_df = DataFrame([dict(bn) for bn in bns])

In [ ]:
if projection_mode == 'node':
    rules: list[tuple[str, list[str]]] = learner.learn_per_node(
        nbsol=0, display=True, lp_solver=lpsolver, max_clause=max_clause,
        max_error=max_error, max_gap=max_gap, timelimit=timelimit,
        subsetmin=optimisation_mode == 'subsetmin'
    )
    max_length = max(len(values) for _, values in rules)
    padded_rules = {
        col: values + [''] * (max_length - len(values))
        for col, values in rules
    }
    rules_df = DataFrame(padded_rules)

## Results

In [ ]:
rules_df